# <center> Deep Dream </center>

<center>This notebook is a short version of https://www.tensorflow.org/tutorials/generative/deepdream</center>
<center>Copyright 2019 The TensorFlow Authors. </center>
<center>Licensed under the Apache License, Version 2.0 </center>

This tutorial contains a minimal implementation of DeepDream, as described in this [blog post](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html) by Alexander Mordvintsev.

DeepDream is an experiment that visualizes the patterns learned by a neural network. Similar to when a child watches clouds and tries to interpret random shapes, DeepDream over-interprets and enhances the patterns it sees in an image.

It does so by forwarding an image through the network, then calculating the gradient of the image with respect to the activations of a particular layer. The image is then modified to increase these activations, enhancing the patterns seen by the network, and resulting in a dream-like image. This process was dubbed "Inceptionism" (a reference to [InceptionNet](https://arxiv.org/pdf/1409.4842.pdf), and the [movie](https://en.wikipedia.org/wiki/Inception) Inception).

Let's demonstrate how you can make a neural network "dream" and enhance the surreal patterns it sees in an image.

In [ ]:
#@title  Import Python libraries

import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image    

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/DlArt/DeepDream/')

from utils import calc_loss, random_roll
from utils import DeepDream, TiledGradients

In [ ]:
#@title Choose an Image from Internet or from Drive 
#@markdown To load file from Drive, upload it first to folder 'DeepDream'

dir = "/content/drive/My Drive/DlArt/DeepDream/"
url = "" #@param {type: "string"}
image_name = "J.jpeg" #@param {type: "string"}

if url:
  name = url.split('/')[-1]
  img_path = tf.keras.utils.get_file(name, origin=url)
elif image_name:
  img_path =  dir + image_name

# Download an image and read it into a NumPy array.
def download(image_path, max_dim=None):
  img = PIL.Image.open(image_path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Normalize an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Display an image
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))


# Downsizing the image makes it easier to work with.
original_img = download(img_path, max_dim=500)
show(original_img)

In [ ]:
#@title Load Pre-trained model


# load model
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
#@title Choose layers 

# Maximize the activations of these layers
names = [ 'mixed1'] #@param {type: "raw"}
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [ ]:
#@title Train model

deepdream = DeepDream(dream_model)
get_tiled_gradients = TiledGradients(dream_model)

# Put all together
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
  base_shape = tf.shape(img)
  img = tf.keras.utils.img_to_array(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  initial_shape = img.shape[:-1]
  img = tf.image.resize(img, initial_shape)
  for octave in octaves:
    # Scale the image based on the octave
    new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
    new_size = tf.cast(new_size, tf.int32)
    img = tf.image.resize(img, new_size)

    for step in range(steps_per_octave):
      gradients = get_tiled_gradients(img, new_size)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1)

      if step % 10 == 0:
        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Octave {}, Step {}".format(octave, step))
    
  result = deprocess(img)
  return result

img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)
base_shape = tf.shape(tf.constant(np.array(original_img)))[:-1]

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)

In [ ]:
#@title Save image

img_name = img_path.split("/")[-1].split(".")[-2]
tf.keras.utils.save_img(dir + img_name + '_' + '-'.join(names) + '.jpeg', img)